In [1]:
import numpy as np
import tensorflow as tf
import datetime
import LoadPatches
from tensorflow import keras
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

%matplotlib inline
%load_ext tensorboard

**Settings**

In [2]:
filename = 'CS8_ROIs.csv'
folder = 'cellsuspensionED8/'
patch_size = 23

# Clipping parameter for histogram equalisation
clip_limit = 0.03


batch_size = 100
epochs = 10

**Load data**

In [3]:
x, y = LoadPatches.get_patches_and_labels(filename, folder, patch_size, clip_limit)

In [4]:
x.dtype, y.dtype

(dtype('float64'), dtype('float64'))

**Create training and validation sets**

In [5]:
def preprocess(x):
    return np.where(x < 128, 1, 0)

def cast(x, y):
    x = tf.cast(x, tf.float32)
    y = tf.cast(y, tf.int64)
    return x, y

def create_dataset(x, y, n_classes=2):
    if n_classes > 2:
        y = tf.one_hot(y, depth=n_classes)
    return tf.data.Dataset.from_tensor_slices((x, y)).map(cast).shuffle(len(y)).batch(100)

In [6]:
X_train, X_val, y_train, y_val = train_test_split(x, preprocess(y), test_size=0.2)

In [7]:
train_ds = create_dataset(X_train, y_train)
val_ds = create_dataset(X_val, y_val)

**Define the model**

In [12]:
# model = keras.Sequential([
#     keras.layers.Conv2D(32, 5, padding='same', activation='relu', input_shape=(patch_size, patch_size, 1)),
#     keras.layers.MaxPooling2D(),
#     keras.layers.Conv2D(64, 5, padding='same', activation='relu'),
#     keras.layers.MaxPooling2D(),
#     keras.layers.Flatten(),
#     keras.layers.Dense(1024, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
# ])
# model.summary()
model = keras.Sequential([
    keras.layers.Reshape(target_shape=(23 * 23,), input_shape=(23, 23)),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=192, activation='relu'),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dense(units=10, activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 529)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               135680    
_________________________________________________________________
dense_3 (Dense)              (None, 192)               49344     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               24704     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
Total params: 211,018
Trainable params: 211,018
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [14]:
log_dir = "logs/" + datetime.datetime.now().strftime("%d%m%Y-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [15]:
model.fit(
    train_ds, 
    epochs=epochs, 
    validation_data=val_ds, 
    validation_steps=2,
    callbacks=[tensorboard_callback]
)

Epoch 1/10
7068/7068 [==============================] - 100s 14ms/step - loss: 0.1917 - accuracy: 0.9607 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
   1/7068 [..............................] - ETA: 76:47:37

KeyboardInterrupt: 